### Servlet Spec
Servlet is a standard for implementing Java classes that respond to requests. Servlets could in principle communicate over any client–server protocol, but they are mostly used with HTTP protocol. The package `javax.servlet` contains all servlet related classes.  

Version history:
```
----------------------------
 Servlet     | Java EE
 Version     | Version
----------------------------
 1.0,2.0,2.1 | -
 2.2         | J2EE 1.2
 2.3         | J2EE 1.3
 2.4         | J2EE 1.4
 2.5         | Java EE 5 
 3.0         | Java EE 6
 3.1         | Java EE 7
 4.0         | Java EE 8
---------------------------- 
```

### Class Hierarchies
```
(I) Servlet 
        |
        +--- (AC) GenericServlet
                            |
                            +--- (C) HttpServlet
```
```
(I) ServletRequest 
        |
        +--- (I) HttpServletRequest
```
```
(I) ServletResponse 
        |
        +--- (I) HttpServletResponse
```
### Lifecycle
Servlet interface defines lifecycle methods
- init(ServletConfig config)
- service(ServletRequest request)
- destroy()

When a servlet container (Apache Tomcat) starts, it creates a `ServletContext` object. The servlet container reads the `web.xml` file and provides all `<context-param>` tags to the context object. 

The container also reads all the `<servlet>`, `<filter>` and `<listener>` tags and creates objects of each. While creating servlet object, its init method is called, passing in a `ServletConfig` object. That object was created by parsing web.xml.  

When a client sends an HTTP request, the servlet container creates new HttpServletRequest and HttpServletResponse objects and passes them through any defined Filter in the chain and, eventually, the Servlet instance. 

In the case of servlets, the service() method is invoked. By default, this method determines which one of the doXxx() methods to invoke based off of request.getMethod().

### Dynamic Web Application
Project structure  
```
webproject
        |
        +--- build
        +--- src
        +--- WebContent
                     |
                     +--- META-INF
                     +--- WEB_INF
                                |
                                +--- lib
                                +--- web.xml (optional, Servlet spec 2.5+)
```

### Servlet
To define a servlet, we add an entry to web.xml,
```xml
<servlet>
    <description>Provides information about the servlet.</description>
    <display-name>InfoServlet</display-name>
    <servlet-name>InfoServlet</servlet-name>
    <servlet-class>com.weblogic.info.InfoServlet</servlet-class>
</servlet>

<servlet-mapping>
    <servlet-name>InfoServlet</servlet-name>
    <url-pattern>/information</url-pattern>
</servlet-mapping>
```

The url pattern can contain wildcards. The corresponding java file is,
```java
public class InfoServlet extends HttpServlet {

    protected void doGet(HttpServletRequest request, HttpServletResponse response) 
        throws ServletException, IOException{
    }

    protected void doPost(HttpServletRequest request, HttpServletResponse response) 
        throws ServletException, IOException{
    }
}    
```

### Servlet Request and Response
To get http request data, we use the `HttpServletRequest` object. For example, to get request header information,
```java
Enumeration<String> reqHeaderNames = request.getHeaderNames();
while(reqHeaderNames.hasMoreElements()) {
    String header = reqHeaderNames.nextElement();
    writer.write( header + " : " + request.getHeader(header) + "<br>");
}
```

To get request parameters,
```java
// Request parameter
writer.write("<br><b>Request Parameters</b><br>");
Enumeration<String> reqParamNames = request.getParameterNames();
while(reqParamNames.hasMoreElements()) {
    String param = reqParamNames.nextElement();
    writer.write( param + " : " + request.getParameter(param) + "<br>");
}
```
If we have a url like http://localhost:8080/WebProject2.5/requestinfo?language=en&location=in, request parameter would be (language, en) and (location,in).  

### Cookies
```java
Cookie[] cookies = request.getCookies();
boolean langCookieExists = false;
if(cookies != null) {
    for(Cookie cookie: cookies) {
        if(cookie.getName().equals("language")) {
            langCookieExists = true;
            break;
        }
    }
}
```

Cookie parameters:
- **cookie.setMaxAge(int expiry)** : sets the expiry time for cookies. If not set, cookie gets destroyed as soon as browser is closed.
- **cookie.setDomain(String pattern)** : this parameter tells the browser that this cookie must only be sent to matching domains. Default behaviour is to send cookies to exactly matching domains (not subdomains). Note that .example.com or even example.com means example.com and all its subdomains. Therefore prefix www into domain name.
- **cookie.setSecure(boolean flag)** : send cookie only over https.
- **cookie.setPath(String uri)** :  specifies a path for the cookie to which the client should return the cookie.

### Sessions
The servlet session implementation uses cookies. When the `HttpSession` is obtained for the first time via request.getSession(), the servlet container creates a new HttpSession object, generates a long and unique ID (which you can get by session.getId()), and stores it in the server's memory. The servlet container also sets a Cookie in the Set-Cookie header of the HTTP response with `JSESSIONID` as its name and the unique session ID as its value.  

The HttpSession stays alive until it has been idle for more than the timeout value specified in `<session-timeout>`, a setting in web.xml. The timeout value defaults to 30 minutes. So, when the client doesn't visit the web app for longer than the time specified, the servlet container trashes the session. Every subsequent request, even with the cookie specified, will not have access to the same session anymore; the servlet container will create a new session.

```java
HttpSession session = request.getSession(); // equivalent to request.getSession(true);
session.setAttribute("username", "max");
```

### Using Annotations
We can use annotations instead of an entry in web.xml. Example:
```java
@WebServlet(
    urlPatterns = {"/annotation"},
    displayName = "Annotation Servlet",         // Optional
    description = "Servlet using annotation"    // Optional
)
class AnnotationServlet extends HttpServlet{
    // ...
```

### ServletContext
In web.xml,
```xml
<!-- Init parameters -->
<context-param>
    <param-name>email</param-name>
    <param-value>admin@test.com</param-value>
</context-param>
  
<context-param>
    <param-name>default language</param-name>
    <param-value>EN</param-value>
</context-param>
```

In servlet java file,
```java
protected void doGet(HttpServletRequest request, HttpServletResponse response) throws ServletException, IOException {
    // Information about ServletContext
    ServletContext context = getServletContext();
    String language = context.getInitParameter("default language");

    Writer writer = response.getWriter();
    writer.write("Context parameter default language = " + language);
}
```

### ServletConfig
To add servlet configuration parameters, in web.xml,
```xml
<servlet>
    <description>Provides information about the servlet.</description>
    <display-name>InfoServlet</display-name>
    <servlet-name>InfoServlet</servlet-name>
    <servlet-class>com.weblogic.info.InfoServlet</servlet-class>
    <init-param>
        <param-name>driver</param-name>
        <param-value>com.oracle.driver.OracleDriver</param-value>
    </init-param>
</servlet>
```

ServletConfig object created by parsing web.xml is passed to the servlet's init() method. We can get access to this object anywhere by calling `getServletConfig()` method.
```java
protected void doGet(HttpServletRequest request, HttpServletResponse response) 
    throws ServletException, IOException {

    ServletConfig config = getServletConfig();
    String driver = config.getInitParameter("driver");
}
```

We can also use annotation notation to configure ServletConfig:
```java
@WebServlet(
    urlPatterns = {"/annotation"},
    displayName = "Annotation Servlet",         
    description = "Servlet using annotation",
    initParams = { @WebInitParam(name = "driver", value = "com.oracle.driver.OracleDriver") } 
)
```